In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.8 MB/s eta 0:00:00


In [ ]:
from transformers import ConvBertTokenizer, ConvBertForMaskedLM
import torch

In [ ]:
def predict_next_word(sentence):
    # Load ConvBERT tokenizer and model
    tokenizer = ConvBertTokenizer.from_pretrained('YituTech/conv-bert-base')
    model = ConvBertForMaskedLM.from_pretrained('YituTech/conv-bert-base')

    # Tokenize input sentence
    tokens = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt')

    # Find the masked token
    masked_index = torch.where(tokens == tokenizer.mask_token_id)[1].tolist()[0]

    # Generate predictions for the masked token
    with torch.no_grad():
        outputs = model(tokens)
        predictions = outputs.logits[0, masked_index]

    # Get the top-k predicted tokens and their probabilities
    top_predictions = torch.topk(predictions, k=5)
    predicted_tokens = tokenizer.convert_ids_to_tokens(top_predictions.indices.tolist())
    probabilities = top_predictions.values.exp().tolist()

    return predicted_tokens, probabilities

In [ ]:
sentence = "I want to [MASK] a car"

predicted_tokens, probabilities = predict_next_word(sentence)
for token, prob in zip(predicted_tokens, probabilities):
    print(f"Token: {token}, Probability: {prob}")

Some weights of ConvBertForMaskedLM were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.weight', 'generator_lm_head.bias', 'generator_predictions.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Token: attendant, Probability: 2424.0068359375
Token: ##grounds, Probability: 463.6900939941406
Token: sergeant, Probability: 363.5892028808594
Token: ##posed, Probability: 301.9273681640625
Token: versa, Probability: 281.9667053222656


## 더정확한 예측 conv-bert-large 사용